In [146]:
# header files
import numpy as np
import torch
import h5py
from torch.utils import data
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.optim as optim

In [138]:
# ImageClassifier Dataset class
class ImageClassifierDataset(data.Dataset):
    # init function
    def __init__(self, file, data_category):
        self.file = file
        self.file_dataset = h5py.File(self.file, 'r')
        
        # get input and output data
        self.file_input_orig = np.array(self.file_dataset[str(data_category) + '_set_x'])
        self.file_output_orig = np.array(self.file_dataset[str(data_category) + '_set_y'])
        self.file_output = self.file_output_orig.reshape(1, self.file_output_orig.shape[0])
        self.file_input = (self.file_input_orig.reshape(self.file_input_orig.shape[0], -1).T) / 255.
        #self.classes = np.array(test_dataset["list_classes"][:])
    
    # length of array for training and testing
    def __len__(self):
        return len(self.file_input[0])
    
    # individual array element for training and testing
    def __getitem__(self, idx):
        return (self.file_input[:, idx], self.file_output[0, idx])
    
    # get input data
    def get_input_data(self):
        return np.array(self.file_input)
    
    # get output data
    def get_output_data(self):
        return np.array(self.file_output)

In [200]:
# get training and testing data
train_file="data/train_catvnoncat.h5"
test_file="data/test_catvnoncat.h5"
train_dataset = ImageClassifierDataset(train_file, "train")
test_dataset = ImageClassifierDataset(test_file, "test")
train_input_data = Variable(torch.from_numpy(train_dataset.get_input_data().astype(np.float32)), requires_grad = True)
test_input_data = Variable(torch.from_numpy(test_dataset.get_input_data().astype(np.float32)), requires_grad = True)
train_output_data = Variable(torch.from_numpy(train_dataset.get_output_data().astype(np.float32)), requires_grad = False)
test_output_data = Variable(torch.from_numpy(test_dataset.get_output_data().astype(np.float32)), requires_grad = False)

In [201]:
# neural network class
class Net(nn.Module):
    # init function
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(12288, 14)
        self.fc2 = nn.Linear(14, 1)
        
    # forward propagation
    def forward(self, x):
        x = F.sigmoid(self.fc2(F.relu(self.fc1(x))))
        return x
    
criterion = nn.CrossEntropyLoss()

In [202]:
net = Net()
optimizer = optim.SGD(net.parameters(), lr=0.01)
print(net)
print(list(net.parameters()))

Net(
  (fc1): Linear(in_features=12288, out_features=14, bias=True)
  (fc2): Linear(in_features=14, out_features=1, bias=True)
)
[Parameter containing:
tensor([[ 0.0032,  0.0042,  0.0086,  ..., -0.0090,  0.0072, -0.0063],
        [-0.0001, -0.0037,  0.0086,  ...,  0.0065, -0.0052,  0.0023],
        [-0.0019, -0.0038, -0.0089,  ...,  0.0065,  0.0056,  0.0037],
        ...,
        [-0.0057, -0.0034, -0.0012,  ..., -0.0039, -0.0042,  0.0009],
        [-0.0084,  0.0066,  0.0075,  ...,  0.0015, -0.0009, -0.0027],
        [ 0.0077, -0.0034, -0.0021,  ...,  0.0039, -0.0045,  0.0026]],
       requires_grad=True), Parameter containing:
tensor([ 0.0058, -0.0006,  0.0046, -0.0052, -0.0023,  0.0040,  0.0003, -0.0020,
         0.0076, -0.0033,  0.0017,  0.0021,  0.0026, -0.0061],
       requires_grad=True), Parameter containing:
tensor([[-0.0375, -0.0889,  0.0563, -0.2121, -0.2052, -0.0856, -0.0061, -0.0668,
          0.0725,  0.1422,  0.2398,  0.2214, -0.0679, -0.1289]],
       requires_grad=True

In [203]:
print(train_output_data)

tensor([[0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 1., 1., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 1., 1., 0., 1., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0.,
         1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 0.,
         0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 0., 1., 1.,
         1., 1., 1., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 1., 0.,
         1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
         1., 1., 1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0.,
         1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1.,
         0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 1.,
         0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.]])


In [210]:
for epoch in range(0, 5):
    optimizer.zero_grad()
    pred_output = []
    for index in range(0, len(train_dataset)):
        pred = net(train_input_data[:, index])
        pred_output.append(np.array([1.0 - pred, pred]))
    pred_output = np.array(pred_output)
    pred_output = Variable(torch.from_numpy(pred_output.astype(np.float32)), requires_grad=False).float()
    loss = criterion(pred_output, torch.tensor([train_output_data[0]]).long())
    loss = Variable(loss, requires_grad=True)
    loss.backward()
    optimizer.step()
    print(loss)

ValueError: only one element tensors can be converted to Python scalars